In [23]:
import requests
from bs4 import BeautifulSoup
import newspaper
from newspaper import Article
import ast
import os
import logging

In [3]:
ANTHROPIC_API_KEY = os.environ['ANTHROPIC_API_KEY']
SERP_API_KEY = os.environ['SERP_API_KEY']

In [4]:
def get_search_terms(topic):
    system_prompt = "You are a world-class journalist. Generate a list of 5 search terms to search for to research and write an article about the topic."
    messages = [
        {"role": "user", 
         "content": f"Please provide a list of 5 search terms related to '{topic}' for researching and writing an article. \
            Respond with the search terms in a Python-parseable list, separated by commas."},
    ]
    headers = {
        "x-api-key": ANTHROPIC_API_KEY,
        "anthropic-version": "2023-06-01",
        "content-type": "application/json"
    }
    data = {
        "model": 'claude-3-haiku-20240307',
        "max_tokens": 200,
        "temperature": 0.5,
        "system": system_prompt,
        "messages": messages,
    }

    response = requests.post("https://api.anthropic.com/v1/messages", headers=headers, json=data)
    response_text = response.json()['content'][0]['text']
    search_terms = ast.literal_eval(response_text)
    return search_terms

In [22]:
topic = 'germany stock market dynamic in april 2024'
search_terms = get_search_terms(topic)
search_terms

['germany stock market performance april 2024',
 'german economy trends april 2024',
 'dax index movement april 2024',
 'factors influencing german stock market april 2024',
 'analysis of german stock market dynamics april 2024']

In [13]:
def get_search_results(search_term):
    url = f"https://serpapi.com/search.json?q={search_term}&api_key={SERP_API_KEY}"
    response = requests.get(url)
    data = response.json()
    return data['organic_results']

In [15]:
search_results = get_search_results(search_terms[0])
search_results

[{'position': 1,
  'title': 'The Term Structure and Interest Rate Dynamics',
  'link': 'https://www.cfainstitute.org/en/membership/professional-development/refresher-readings/term-structure-interest-rate-dynamics',
  'redirect_link': 'https://www.google.com/url?sa=t&source=web&rct=j&opi=89978449&url=https://www.cfainstitute.org/en/membership/professional-development/refresher-readings/term-structure-interest-rate-dynamics&ved=2ahUKEwiZ_cfmydOFAxUxSzABHcDfAV0QFnoECBUQAQ',
  'displayed_link': 'https://www.cfainstitute.org › refresher-readings › term-...',
  'favicon': 'https://serpapi.com/searches/66252a901e803f98a03cc69e/images/e52c14ab22f91149097e69bdf0c14304a7233ff50d5412415965322031f61df6.png',
  'snippet': 'This dynamic implies an expected return in excess of short-maturity bonds (i.e., a term premium) for longer-maturity bonds if the yield curve is upward sloping.',
  'snippet_highlighted_words': ['implies an expected return in excess of short-maturity bonds'],
  'source': 'CFA Ins

In [24]:
def select_relevant_urls(search_results):
    system_prompt = "You are a journalist assistant. \
        From the given search results, select the URLs that seem most relevant and informative \
            for writing an article on the topic."
    search_results_text = "\n".join([f"{i+1}. {result['link']}" for i, result in enumerate(search_results)])
    messages = [
        {"role": "user", 
         "content": f"Search Results:\n{search_results_text}\n\n\
            Please select the numbers of the URLs that seem most relevant and informative for writing an article on the topic. \
                Respond with the numbers in a Python-parseable list, separated by commas."},
    ]
    headers = {
        "x-api-key": ANTHROPIC_API_KEY,
        "anthropic-version": "2023-06-01",
        "content-type": "application/json"
    }
    data = {
        "model": 'claude-3-haiku-20240307',
        "max_tokens": 200,
        "temperature": 0.5,
        "system": system_prompt,
        "messages": messages,
    }
    response = requests.post("https://api.anthropic.com/v1/messages", headers=headers, json=data)
    response_text = response.json()['content'][0]['text']

    try:
        numbers = ast.literal_eval(response_text)
        relevant_indices = [int(num) - 1 for num in numbers]
        relevant_urls = [search_results[i]['link'] for i in relevant_indices]
        logging.info(relevant_urls)
        return relevant_urls
    except:
        return []

# Perform searches and select relevant URLs
relevant_urls = []
for term in search_terms:
    logging.info(f'Searching for {term} ...')
    search_results = get_search_results(term)
    urls = select_relevant_urls(search_results)
    relevant_urls.extend(urls)

unique_relevant_urls = list(set(relevant_urls))
unique_relevant_urls

['https://www.cnbc.com/2024/02/21/germany-slashes-2024-growth-forecast-to-just-0point2percent-amid-slower-than-expected-rebound.html',
 'https://www.dw.com/en/german-central-bank-warns-of-recession-citing-strikes/a-68300874',
 'https://www.cnbc.com/2024/04/17/european-markets-live-updates-stocks-news-data-and-earnings.html',
 'https://capex.com/en/overview/dax-40-forecast-price-predictions',
 'https://markets.ft.com/data/world/countries/germany',
 'https://www.imf.org/en/News/Articles/2024/04/16/tr041624-transcript-of-april-2024-weo-press-briefing',
 'https://www.statista.com/statistics/261678/annual-dax-trends-since-1987/',
 'https://www.rolandberger.com/en/Insights/Publications/German-economic-outlook-in-2024.html',
 'https://www.statista.com/statistics/1104490/weekly-dax-index-performance/',
 'https://economy-finance.ec.europa.eu/economic-surveillance-eu-economies/germany/economic-forecast-germany_en',
 'https://www.reuters.com/markets/europe/germanys-economic-advisers-cut-2024-grow

In [26]:
def get_article_text(url):
    article = Article(url)
    article.download()
    article.parse()
    return article.text

# Get article text from relevant URLs
article_texts = []
for url in unique_relevant_urls:
  try:
    text = get_article_text(url)
    if len(text) > 75:
      print(f'Ok for {url}')
      article_texts.append(text)
  except:
    pass

print('Articles to reference:', article_texts)

Ok for https://www.cnbc.com/2024/02/21/germany-slashes-2024-growth-forecast-to-just-0point2percent-amid-slower-than-expected-rebound.html
Ok for https://www.dw.com/en/german-central-bank-warns-of-recession-citing-strikes/a-68300874
Ok for https://www.cnbc.com/2024/04/17/european-markets-live-updates-stocks-news-data-and-earnings.html
Ok for https://capex.com/en/overview/dax-40-forecast-price-predictions
Ok for https://markets.ft.com/data/world/countries/germany
Ok for https://www.imf.org/en/News/Articles/2024/04/16/tr041624-transcript-of-april-2024-weo-press-briefing
Ok for https://www.statista.com/statistics/261678/annual-dax-trends-since-1987/
Ok for https://www.rolandberger.com/en/Insights/Publications/German-economic-outlook-in-2024.html
Ok for https://www.statista.com/statistics/1104490/weekly-dax-index-performance/
Ok for https://economy-finance.ec.europa.eu/economic-surveillance-eu-economies/germany/economic-forecast-germany_en
Ok for https://www.imf.org/en/News/Articles/2024/04

In [27]:
def write_article(topic, article_texts):
    system_prompt = "You are a journalist. \
        Write a high-quality, NYT-worthy article on the given topic based on the provided article texts. \
            The article should be well-structured, informative, and engaging."
    combined_text = "\n\n".join(article_texts)
    messages = [
        {"role": "user", 
         "content": f"Topic: {topic}\n\nArticle Texts:\n{combined_text}\n\n\
            Please write a high-quality, NYT-worthy article on the topic based on the provided article texts. \
                The article should be well-structured, informative, and engaging. \
                    Ensure the length is at least as long as a NYT cover story -- at a minimum, 15 paragraphs."},
    ]
    headers = {
        "x-api-key": ANTHROPIC_API_KEY,
        "anthropic-version": "2023-06-01",
        "content-type": "application/json"
    }
    data = {
        "model": 'claude-3-opus-20240229',
        "max_tokens": 3000,
        "temperature": 0.5,
        "system": system_prompt,
        "messages": messages,
    }
    response = requests.post("https://api.anthropic.com/v1/messages", headers=headers, json=data)
    article = response.json()['content'][0]['text']
    return article

article = write_article(topic, article_texts)
print("\nGenerated Article:")
print(article)




Generated Article:
Germany's Economy Stumbles as DAX Soars to New Heights

BERLIN — Germany, Europe's largest economy, is facing a perplexing dichotomy. While the country teeters on the brink of a technical recession, its benchmark stock index, the DAX, has been setting record highs. This stark contrast between the struggling domestic economy and the booming stock market has economists and investors puzzled.

In April 2024, German Economy Minister Robert Habeck slashed the country's growth forecast for the year to a meager 0.2%, a significant drop from the previous estimate of 1.3%. Habeck attributed this downward revision to an unstable global economic environment, historically low global trade growth, and the challenges these pose for an export-driven nation like Germany.

The Bundesbank, Germany's central bank, has warned that economic output is set to fall again in the first quarter of 2024, pushing the country into a technical recession, defined as two consecutive quarters of neg

In [28]:
def edit_article(article):
    system_prompt = "You are an editor. \
        Review the given article and provide suggestions for improvement. \
            Focus on clarity, coherence, and overall quality."
    messages = [
        {"role": "user", 
         "content": f"Article:\n{article}\n\n\
            Please review the article and provide suggestions for improvement. \
                Focus on clarity, coherence, and overall quality."},
    ]
    headers = {
        "x-api-key": ANTHROPIC_API_KEY,
        "anthropic-version": "2023-06-01",
        "content-type": "application/json"
    }
    data = {
        "model": 'claude-3-opus-20240229',
        "max_tokens": 3000,
        "temperature": 0.5,
        "system": system_prompt,
        "messages": messages,
    }
    response = requests.post("https://api.anthropic.com/v1/messages", headers=headers, json=data)
    suggestions = response.json()['content'][0]['text']

    system_prompt = "You are an editor. Rewrite the given article based on the provided suggestions for improvement."
    messages = [
        {"role": "user", "content": f"Original Article:\n{article}\n\nSuggestions for Improvement:\n{suggestions}\n\nPlease rewrite the article based on the provided suggestions for improvement."},
    ]
    data = {
        "model": 'claude-3-opus-20240229',
        "max_tokens": 3000,
        "temperature": 0.5,
        "system": system_prompt,
        "messages": messages,
    }
    response = requests.post("https://api.anthropic.com/v1/messages", headers=headers, json=data)
    edited_article = response.json()['content'][0]['text']
    return edited_article

edited_article = edit_article(article)
print("\nEdited Article:")
print(edited_article)


Edited Article:
Germany's Economy Stumbles as DAX Soars to New Heights

BERLIN — Germany, Europe's largest economy, is facing a perplexing dichotomy. While the country teeters on the brink of a technical recession, its benchmark stock index, the DAX, has been setting record highs. This stark contrast between the struggling domestic economy and the booming stock market has economists and investors puzzled.

In April 2024, German Economy Minister Robert Habeck slashed the country's growth forecast for the year to a meager 0.2%, a significant drop from the previous estimate of 1.3%. Habeck attributed this downward revision to an unstable global economic environment and historically low global trade growth, which pose challenges for an export-driven nation like Germany.

The Bundesbank, Germany's central bank, has warned that economic output is set to fall again in the first quarter of 2024, pushing the country into a technical recession, defined as two consecutive quarters of negative gr